In [152]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

base_url = "https://www.dubizzle.com.eg/en/properties/apartments-duplex-for-sale/?page={}"
apartments_links = []

page = 1
while  True:
    print(f"\nProcessing page {page}...")
    url = base_url.format(page)
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 403:
            print("Access denied (403). Possibly blocked by website.")
            break
    except Exception as e:
        print(f"Failed to load page {page}: {e}")
        break

    soup_list = BeautifulSoup(response.text, 'html.parser')
    apartments = soup_list.find_all('div', class_='_637fa00f')

    if not apartments:
        print("No apartments found on this page. Stopping.")
        break

    print(f"Found {len(apartments)} apartments on page {page}.")
    for apartment in apartments:
        try:
            link = apartment.find('a', href=True)['href']
            if not link.startswith('http'):
                full_link = f"https://www.dubizzle.com.eg{link}"
            else:
                full_link = link
            apartments_links.append(full_link)
        except Exception as e:
            print(f"Failed to extract link: {e}")
            continue

    page += 1
    time.sleep(1)

# Remove duplicate links
apartments_links = list(set(apartments_links))
print(f"\nTotal {len(apartments_links)} unique links collected.\n")

data = []

for idx, link in enumerate(apartments_links, 1):
    property_details = {
        'link': link,
        'title': 'Not Available',
        'price': 'Not Available',
        'down_payment': 'Not Available',
        'location': 'Not Available',
        'creation_date': 'Not Available',
        'property_type': 'Not Available',
        'ownership': 'Not Available',
        'area': 'Not Available',
        'bedrooms': 'Not Available',
        'bathrooms': 'Not Available',
        'furnished': 'Not Available',
        'level': 'Not Available',
        'payment_option': 'Not Available',
        'completion_status': 'Not Available',
        'amenities': 'Not Available',
        'ad_id': 'Not Available',
        'seller_type': 'Not Available',
        'seller_name': 'Not Available',
        'seller_member_since': 'Not Available',
    }

    try:
        response = requests.get(link, headers=headers, timeout=10)
        response.raise_for_status()
        soup_detail = BeautifulSoup(response.content, 'html.parser')

        # Title
        title = soup_detail.find('h1', class_='_75bce902')
        if title:
            property_details['title'] = title.text.strip()

        # Price
        price = soup_detail.find('span', string=lambda x: x and "EGP" in x)
        if price:
            property_details['price'] = price.text.strip()

        # Down payment
        down_payment = soup_detail.find('span', class_='_2ca33d5e', attrs={'aria-label': 'Price Info'})
        if down_payment:
            property_details['down_payment'] = down_payment.text.strip()

        # Location
        location = soup_detail.find('span', class_='a1c1940e', attrs={'aria-label': 'Location'})
        if location:
            property_details['location'] = location.text.strip()

        # Creation date
        creation_date = soup_detail.find('span', attrs={'aria-label': 'Creation date'})
        if creation_date:
            property_details['creation_date'] = creation_date.text.strip()

        # Highlighted Details
        highlights = soup_detail.find('div', attrs={'aria-label': 'Highlighted Details'})
        if highlights:
            details = highlights.find_all('div', class_='_2ea1161b')
            for detail in details:
                label = detail.find('span', class_='_8206696c')
                value = detail.find('span', class_='b7af14b4')
                if label and value:
                    label_text = label.text.strip()
                    value_text = value.text.strip()
                    if label_text == 'Type':
                        property_details['property_type'] = value_text
                    elif label_text == 'Ownership':
                        property_details['ownership'] = value_text
                    elif label_text == 'Area (m²)':
                        property_details['area'] = value_text
                    elif label_text == 'Bedrooms':
                        property_details['bedrooms'] = value_text
                    elif label_text == 'Bathrooms':
                        property_details['bathrooms'] = value_text
                    elif label_text == 'Furnished':
                        property_details['furnished'] = value_text

        # Other Details
        details_section = soup_detail.find('div', attrs={'aria-label': 'Details'})
        if details_section:
            detail_items = details_section.find_all('div', class_='_9a8eacd9')
            for item in detail_items:
                spans = item.find_all('span')
                if len(spans) == 2:
                    label = spans[0].text.strip()
                    value = spans[1].text.strip()
                    if label == 'Level':
                        property_details['level'] = value
                    elif label == 'Payment Option':
                        property_details['payment_option'] = value
                    elif label == 'Completion status':
                        property_details['completion_status'] = value

        # Amenities
        amenities_section = soup_detail.find('div', attrs={'aria-label': 'Features'})
        if amenities_section:
            amenities = amenities_section.find_all('span', class_=['c327b807', '_241f82d9'])
            amenities_list = [amenity.text.strip() for amenity in amenities]
            if amenities_list:
                property_details['amenities'] = ', '.join(amenities_list)

        # Ad ID
        ad_id = soup_detail.find('div', class_='_114d5a00')
        if ad_id:
            property_details['ad_id'] = ad_id.text.strip().replace('Ad id ', '')

        # Seller Info
        seller_section = soup_detail.find('div', attrs={'aria-label': 'Seller description'})
        if seller_section:
            seller_type = seller_section.find('span', class_='e74aa2ac')
            if seller_type:
                property_details['seller_type'] = seller_type.text.strip()

            seller_name = seller_section.find('span', class_=lambda x: x and ('_9a85fb36' in x and 'b7af14b4' in x) or ('_8206696c' in x and 'b7af14b4' in x))
            if seller_name and 'See profile' not in seller_name.text:
                property_details['seller_name'] = seller_name.text.strip()

            seller_member_since = seller_section.find('span', class_='_8206696c', string=lambda x: 'Member since' in x if x else False)
            if seller_member_since:
                property_details['seller_member_since'] = seller_member_since.text.strip()

        print(f"[{idx}/{len(apartments_links)}] Extracted: {link}")
        data.append(property_details)
        time.sleep(1)

        # Save backup every 5 entries
        if idx % 5 == 0:
            backup_path = "backup.csv"
            pd.DataFrame(data).to_csv(backup_path, index=False, encoding='utf-8-sig')
            print(f"Backup saved at {backup_path}")

    except Exception as e:
        print(f"Error extracting {link}: {e}")
        for key in property_details:
            if key != 'link' and property_details[key] == 'Not Available':
                property_details[key] = 'Error'
        data.append(property_details)

# Save final result
output_path = r'E:\Computing&Data Science\Semester 6\Tools\final project\Dubizzle_properties.csv'
df = pd.DataFrame(data)
df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"\nData successfully saved to: {output_path}")



Processing page 1...
Found 45 apartments on page 1.

Processing page 2...
Found 45 apartments on page 2.

Processing page 3...
Found 45 apartments on page 3.

Processing page 4...
Found 45 apartments on page 4.

Processing page 5...
Found 45 apartments on page 5.

Processing page 6...
Found 45 apartments on page 6.

Processing page 7...
Found 45 apartments on page 7.

Processing page 8...
Found 45 apartments on page 8.

Processing page 9...
Found 45 apartments on page 9.

Processing page 10...
Found 45 apartments on page 10.

Processing page 11...
Found 45 apartments on page 11.

Processing page 12...
Found 45 apartments on page 12.

Processing page 13...
Found 45 apartments on page 13.

Processing page 14...
Found 45 apartments on page 14.

Processing page 15...
Found 45 apartments on page 15.

Processing page 16...
Found 45 apartments on page 16.

Processing page 17...
Found 45 apartments on page 17.

Processing page 18...
Found 45 apartments on page 18.

Processing page 19...
Found